### **The EDA of Physionet Data set regarding "A Large Scale 12 Lead Electrocardiogram Database for Arrhythmia Study 1.0.0"**

- **Imports**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import wfdb
import os
from scipy.signal import butter, filtfilt
import biosppy.signals.ecg as ecg
import psycopg2
from sklearn.decomposition import IncrementalPCA

- **Declaring Global variables**

In [ ]:
dataset_path = r'..\a-large-scale-12-lead-electrocardiogram-database-for-arrhythmia-study-1.0.0\WFDBRecords\\'
count = 0
lowcut = 0.66  # Lower cutoff frequency in Hz
highcut = 46.0   # Higher cutoff frequency in Hz
dbconn = None
dbcur = None

- **Visual each ECG Signal using wfdb function**

In [ ]:
def visualize_data(record):
    _,axes = plt.subplots(12,1,figsize=(12,20))
    leads = ["I", "II", "III", "avR", "avF", "aVL", "V1", "V2", "V3", "V4", "V5", "V6"]  
    for i, lead in enumerate(leads):
        axes[i].plot(record[:, i])
        axes[i].set_ylabel(lead)       
    plt.legend(leads)
    plt.show()

- **Filtering Noise and Normalizing Signals**

In [ ]:
# Define a Butterworth bandpass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    # Calculate the Nyquist frequency
    nyquist_freq = 0.5 * fs
    # Normalize the cutoff frequencies
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    # Design the Butterworth bandpass filter
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a

# Apply the bandpass filter to the input data
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    # Design the Butterworth bandpass filter
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    # Filter the input data using the designed filter
    filtered_data = filtfilt(b, a, data)
    return filtered_data

- **Calculating Heart Rate**

In [ ]:
def calculate_heartrate(record,fs):
    ecg_signal = record[:,1]
    # Process ECG signal to detect R-peaks
    rpeaks = ecg.engzee_segmenter(ecg_signal, sampling_rate=500)[0]
    # Calculate RR intervals (in seconds)
    rr_intervals = np.diff(rpeaks) / fs
    # Calculate Heart rate of Pateint
    heart_rate = 60 * len(rr_intervals) / np.sum(rr_intervals)
    return heart_rate

- **Calculating QRS Interval**

In [ ]:
def calculate_qrs_interval(record,fs):
    ecg_signal = record[:,1]
    # Process ECG signal to detect QRS complexes
    qrs_indices = ecg.hamilton_segmenter(ecg_signal, sampling_rate=500)[0]

    qrs_durations = []
    for qrs_index in qrs_indices:
        qrs_start = max(0, qrs_index - 0.05 * fs)
        qrs_end = min(len(ecg_signal), qrs_index + 0.05 * fs)
        qrs_duration = (qrs_end - qrs_start) / float(fs)
        qrs_durations.append(qrs_duration)
    avg_qrs_duration = np.mean(qrs_durations)
    formatted_avg_qrs_duration = "{:.3f}".format(avg_qrs_duration)
    return formatted_avg_qrs_duration

### **Applying Partial Principle Component Analysis**

In [ ]:
def visualize_pca_incremental(data_generator, batch_size=2000):
    # Initialize Incremental PCA
    ipca = IncrementalPCA(n_components=12)
    # Process data in batches
    for batch_data in data_generator(batch_size=batch_size):
        print(len(batch_data))
        total_data = np.empty((batch_size * 5000, 12))
        for i, data_array in enumerate(batch_data):
            start_index = i * 5000
            end_index = start_index + 5000
            total_data[start_index:end_index] = data_array
        
        centered_data = total_data - np.mean(total_data, axis=0)
        
        ipca.partial_fit(centered_data)
        batch_data.clear()


    # Get the principal components (eigenvectors)
    principal_components = ipca.components_

    # Get explained variance ratios
    explained_variance = ipca.explained_variance_ratio_

    # Plot explained variance for analysis (optional)
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(explained_variance) + 1), explained_variance, marker='o', linestyle='-')
    plt.xlabel('Principal Component')
    plt.ylabel('Explained Variance Ratio')
    plt.title('Explained Variance by Principal Components')
    plt.grid(True)
    plt.show()

    threshold = 0.9  # Example threshold: retain 90% of explained variance

    explained_variance_sum = np.sum(explained_variance)
    num_components = 0
    for i in range(len(explained_variance)):
        explained_variance_sum += explained_variance[i]
        num_components = i + 1
        if explained_variance_sum >= threshold:
            break

    print(f"Number of components to retain for {threshold*100:.0f}% explained variance:", num_components)

    # Select the chosen number of principal components
    selected_components = principal_components[:, :num_components]

## Can Play Aroung witht Components or Can do futher processing
    

In [ ]:
def data_generator(batch_size=2000):

    batch_data = []
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="EDA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    try:
        # Fetch data from the database in batches
        offset = 0
        while True:
            # Fetch up to batch_size entries from the database
            cur.execute(f"SELECT n_signal FROM ECG LIMIT {batch_size} OFFSET {offset}")
            rows = cur.fetchall()
            
            if not rows:
                break  # No more data to fetch
            
            # Convert fetched data to numpy array
            for row in rows:
                data = np.frombuffer(row[0],dtype=float).reshape(-1, 12)
                # data = data.
                
                batch_data.append(data)

            yield batch_data

            # Move to the next batch
            offset += batch_size

    finally:
        # Close database connection
        cur.close()
        conn.close()



### **Data Insertion**

In [ ]:
def data_Insertor():
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="EDA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    
    try:
        createQuery = '''CREATE TABLE IF NOT EXISTS ECG(
                ID varchar(10) primary key,
                AGE int,
                GENDER varchar(10),
                DX TEXT,
                HR REAL,
                p_signal BYTEA,
                n_signal BYTEA)'''
                
                # QRS REAL,
                # NGraph BYTEA,
                # OGraph BYTEA,
        cursor.execute(createQuery)
            
        for dir1 in os.listdir(dataset_path):
            dir1_path = os.path.join(dataset_path, dir1)
            
            # Loop through the subdirectories
            for dir2 in os.listdir(dir1_path):
                dir2_path = os.path.join(dir1_path, dir2)

                # Loop through the ECG records
                for file_name in os.listdir(dir2_path):
                    if file_name.endswith('.mat'):
                        file_path = os.path.join(dir2_path, file_name[:-4])
                        record = wfdb.rdrecord(file_path)
                        
                        write_to_db(conn, cursor, record)

    
    finally:
        # Close database connection
        cursor.close()
        conn.close()

def write_to_db(conn, cursor, record):

    try:
        age = int(record.comments[0][5:])
    except:
        age = 62
                        
    # o_plot_data = visualize_data(record.p_signal)
    filtered_p_signal = np.zeros_like(record.p_signal)
    for lead in range(12):
        filtered_signal = bandpass_filter(record.p_signal[:, lead], lowcut, highcut, 500)
        filtered_p_signal[:, lead] = filtered_signal
                        
    # n_plot_data = visualize_data(filtered_p_signal)
                                                
    insertValues = (record.record_name,
                    age,
                    record.comments[1][5:],
                    record.comments[2][4:],
                    calculate_heartrate(filtered_p_signal,record.fs),
                    # calculate_qrs_interval(filtered_p_signal,record.fs),
                    # psycopg2.Binary(n_plot_data),
                    # psycopg2.Binary(o_plot_data),
                    psycopg2.Binary(record.p_signal.tobytes()),
                    psycopg2.Binary(filtered_p_signal.tobytes()))
    cursor.execute('''INSERT INTO ECG (ID, AGE, GENDER,DX,HR,p_signal,n_signal) 
                   VALUES (%s,%s,%s,%s,%s,%s,%s)''',insertValues)
    
    conn.commit()


- **Adding Diagnosis Count to DB**

In [ ]:
def Alter_table():
    conn = psycopg2.connect(
        
        dbname="EDA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    
    cursor = conn.cursor()
    
    cursor.execute("Alter Table ECG add column dxcount int")
    
    conn.commit()
    
    cursor.close()
    conn.close()
    
    
    
Alter_table() 
            

- **Saving Acronyns to DB**

In [ ]:
from Utils import process_dx
def write_acronym_to_db():
# def write_pca_to_db(selected_components):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        
        dbname="EDA",
        user="postgres",

        password="admin",
        host="localhost",
        port="5432"
    )
    
    cursor = conn.cursor()
    try:
        cursor.execute("SELECT id,dx FROM ECG")
        dx  = cursor.fetchall()
        for id,dxcode in dx:
            cursor.execute("UPDATE ECG SET dxname= %s WHERE id = %s", (process_dx(dxcode), id))
            conn.commit() 
    finally:
        cursor.close()
        conn.close() 
        
write_acronym_to_db()  

- **Droping Columns to Push to CockRoachDB**

In [ ]:
'''
The Database for this set is about 44 GB in size but Cockroach DB free teir only provide 10 GB, so I have to drop Columns or Two to Manage Size
'''
import psycopg2

def drop_columns():
    try:
        conn = psycopg2.connect(
            dbname="EDA",
            user="postgres",
            password="admin",
            host="localhost",
            port="5432"
        )

        cursor = conn.cursor()

        # Drop the columns dxname and acronym
        cursor.execute("Alter Table ECG Drop Column n_signal, p_signal")

        conn.commit()

        print("Columns dropped successfully.")

    except (Exception, psycopg2.Error) as error:
        print("Error while dropping columns:", error)

    finally:
        if conn:
            cursor.close()
            conn.close()

drop_columns()

- **Inserting Selected Coulmn to CockRoachDB**

In [ ]:
import os
from Utils import process_dx
import psycopg2

os.environ["DATABASE_URL"] = "YOUR LINK HERE"

def write_to_db(cursor, record):
    try:
        age = int(record.comments[0][5:])
    except:
        age = 62
                        
    dxcode = record.comments[2][4:]
    
    # Check if ID already exists in the database
    cursor.execute("SELECT COUNT(*) FROM ECG WHERE ID = %s", (record.record_name,))
    result = cursor.fetchone()[0]

    # If the ID doesn't exist, insert the record
    if result == 0:
        insertValues = (record.record_name,
                        age,
                        record.comments[1][5:],
                        dxcode,
                        calculate_heartrate(record.p_signal,record.fs),
                        len(dxcode.split(',')),
                        process_dx(dxcode)
                        )
        cursor.execute('''INSERT INTO ECG (ID, AGE, GENDER, DX, HR, dxcount, dxname) 
                        VALUES (%s,%s,%s,%s,%s,%s,%s)''', insertValues)
        print(f"Record '{record.record_name}' inserted into the database.")
    else:
        print(f"Record '{record.record_name}' already exists in the database. Skipping insertion.")
        

# Establish database connection
conn = psycopg2.connect(os.environ["DATABASE_URL"])
counter = 0 
# Execute the insertion process
cursor = conn.cursor()
for dir1 in os.listdir(dataset_path):
        dir1_path = os.path.join(dataset_path, dir1)
        
        # Loop through the subdirectories
        for dir2 in os.listdir(dir1_path):
            dir2_path = os.path.join(dir1_path, dir2)

            # Loop through the ECG records
            for file_name in os.listdir(dir2_path):
                if file_name.endswith('.mat'):
                    if counter > 10990:
                            file_path = os.path.join(dir2_path, file_name[:-4])
                            record = wfdb.rdrecord(file_path)
                            write_to_db(cursor, record)
                    counter+=1
                    print(counter)
        if counter > 10990:
            conn.commit()
cursor.close()
conn.close()

- **Deleting UnRequired Data Entries**

In [ ]:
import psycopg2
import os
os.environ["DATABASE_URL"] = "Put YOUr LINK HERE"

def gender_deletor():

    # Connect to the PostgreSQL database
    conn = psycopg2.connect(os.environ["DATABASE_URL"])
    cur = conn.cursor()

    try:
        # Fetch up to batch_size entries from the database
        cur.execute(f"SELECT id,gender FROM ECG")
        rows = cur.fetchall()
        
        # Convert fetched data to numpy array
        for row in rows:
            
            # print(f"{row[0]}: {row[1]}")
            if row[1] == "Unknown":
                # with open("gender_deleted.txt","a") as f:
                #     f.write(f"{row[0]}\n")
                cur.execute(f"DELETE FROM ECG WHERE id = '{row[0]}'")
        conn.commit()

    finally:
        cur.close()
        conn.close()

gender_deletor()

In [ ]:
import psycopg2
import os
os.environ["DATABASE_URL"] = "Put YOUr LINK HERE"

def age_deletor():

    conn = psycopg2.connect(os.environ["DATABASE_URL"])
    cur = conn.cursor()
    cur = conn.cursor()

    try:
        # Fetch up to batch_size entries from the database
        cur.execute(f"SELECT id,age FROM ECG")
        rows = cur.fetchall()
        
        # Convert fetched data to numpy array
        for row in rows:
            
            # print(f"{row[0]}: {row[1]}")
            if row[1] == 0:
                with open("age_deleted.txt","a") as f:
                    f.write(f"{row[0]}\n")
                # cur.execute(f"DELETE FROM ECG WHERE id = '{row[0]}'")
                # conn.commit()

    finally:
        # Close database connection
        cur.close()
        conn.close()
        
age_deletor()
              

In [ ]:
def signal_deletor(batch_size=2000):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(
        dbname="EDA",
        user="postgres",
        password="admin",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    try:
        # Fetch data from the database in batches
        offset = 0
        while True:
            # Fetch up to batch_size entries from the database
            cur.execute(f"SELECT id,p_signal FROM ECG LIMIT {batch_size} OFFSET {offset}")
            rows = cur.fetchall()
            
            if not rows:
                break  # No more data to fetch
            
            # Convert fetched data to numpy array
            for row in rows:
                data = np.frombuffer(row[1],dtype=float)
                data = data.reshape((5000,12))

                print(f"{row[0]}: {np.sum(np.isnan(data))}")
                if(np.sum(np.isnan(data)) > 0):
                    with open("status.txt","a") as f:
                        f.write(f"{row[0]}: {np.sum(np.isnan(data))}\n")
                    
                    #delete from database
                    
                    # cur.execute(f"DELETE FROM ECG WHERE id = '{row[0]}'")
                    # conn.commit()

            # Move to the next batch
            offset += batch_size

    finally:
        # Close database connection
        cur.close()
        conn.close()

signal_deletor(2000)